## <font size=4> *Modelación y Simulación*, 2024 </font>
## <font size=3 color='gray'> Adrian Rodríguez</font>

In [14]:
import Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")
Pkg.add("DataFrames")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [15]:
using JuMP
using HiGHS
using DataFrames

months = 1:6
demand = [180, 250, 190, 140, 220, 250]
production_cost = [50, 45, 55, 52, 48, 50]
inventory_cost = [8, 10, 10, 10, 8, 8]
capacity = 225


model = Model(HiGHS.Optimizer)

@variable(model, x[months] >= 0, start=0)  # x[m] = la cantidad de ventanas producidas en el mes m
@variable(model, I[months] >= 0, start=0)  # I[m] = inventario al final del mes m

# Capacidad de producción
@constraint(model, [m in months], x[m] <= capacity)

# Balance de inventario
@constraint(model, I[1] == x[1] - demand[1])
@constraint(model, [m in 2:6], I[m] == I[m-1] + x[m] - demand[m])  

# Minimizar el costo total, que es la suma de los costos de producción y de inventario
@objective(model, Min, 
    sum(production_cost[m] * x[m] + inventory_cost[m] * I[m] for m in months)
)


optimize!(model)


Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+00, 6e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+02]
Solving LP without presolve, or with basis, or unconstrained
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 6(1230) 0s
         13     6.1795000000e+04 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 13
Objective value     :  6.1795000000e+04
HiGHS run time      :          0.00


In [16]:
status = termination_status(model)

if status == MOI.OPTIMAL
    println("Optimal solution found.\n")
    
    production_values = collect(value.(x))
    inventory_values = collect(value.(I))
    
    direct_production_cost = [demand[i] * production_cost[i] for i in months] # asumiendo que se puede producir exactamente la demanda
    optimal_production_cost = [production_values[i] * production_cost[i] for i in months]

    results = DataFrame(
        :Month => months,
        :Production => production_values,
        :Inventory => inventory_values,
        :Optimal_Production_Cost =>  optimal_production_cost,
        :Direct_Production_Cost => direct_production_cost,
        :Difference =>   optimal_production_cost .- direct_production_cost
    )

    println(results)
    println("\nTotal cost: \$", objective_value(model))
    println("Naive production cost: \$", sum(direct_production_cost))
    println("Cost difference: \$", sum(results.Difference))
    
else
    println("Optimization was not successful. Status: ", status)
end


Optimal solution found.

6×6 DataFrame
 Row │ Month  Production  Inventory  Optimal_Production_Cost  Direct_Production_Cost  Difference 
     │ Int64  Float64     Float64    Float64                  Int64                   Float64    
─────┼───────────────────────────────────────────────────────────────────────────────────────────
   1 │     1       205.0       25.0                  10250.0                    9000      1250.0
   2 │     2       225.0        0.0                  10125.0                   11250     -1125.0
   3 │     3       190.0        0.0                  10450.0                   10450         0.0
   4 │     4       160.0       20.0                   8320.0                    7280      1040.0
   5 │     5       225.0       25.0                  10800.0                   10560       240.0
   6 │     6       225.0        0.0                  11250.0                   12500     -1250.0

Total cost: $61795.0
Naive production cost: $61040
Cost difference: $155.0


### Versión con restricciones enteras

In [17]:
model_integer = Model(HiGHS.Optimizer)

@variable(model_integer, x[months] >= 0, Int)  # Producción entera en cada mes
@variable(model_integer, I[months] >= 0, Int)  # Inventario entero al final de cada mes

@constraint(model_integer, [m in months], x[m] <= capacity)

@constraint(model_integer, I[1] == x[1] - demand[1])
@constraint(model_integer, [m in 2:6], I[m] == I[m-1] + x[m] - demand[m])

@objective(model_integer, Min, 
    sum(production_cost[m] * x[m] + inventory_cost[m] * I[m] for m in months)
)

optimize!(model_integer)



Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+00, 6e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+02]
Presolving model
5 rows, 11 cols, 15 nonzeros  0s
3 rows, 9 cols, 11 nonzeros  0s
1 rows, 3 cols, 3 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      61795
  Dual bound        61795
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    61795 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (total)
                    0.00 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)


In [19]:
status_integer = termination_status(model_integer)

if status_integer == MOI.OPTIMAL
    println("\nSolución óptima encontrada (Variables Enteras).\n")
    
    production_values_int = [value(x[m]) for m in months]
    inventory_values_int = [value(I[m]) for m in months]
    
    direct_production_cost_int = demand .* production_cost
    
    results_integer = DataFrame(
        Month = collect(months),
        Production = production_values_int,
        Inventory = inventory_values_int,
        Optimal_Production_Cost = production_values_int .* production_cost,
        Direct_Production_Cost = direct_production_cost_int
    )
    
    println(results_integer)
    println("\nTotal cost: \$", objective_value(model))
    println("Naive production cost: \$", sum(direct_production_cost))
    println("Cost difference: \$", sum(results.Difference))
    
else
    println("La optimización no fue exitosa (Entero). Estado: ", status_integer)
end


Solución óptima encontrada (Variables Enteras).

6×5 DataFrame
 Row │ Month  Production  Inventory  Optimal_Production_Cost  Direct_Production_Cost 
     │ Int64  Float64     Float64    Float64                  Int64                  
─────┼───────────────────────────────────────────────────────────────────────────────
   1 │     1       205.0       25.0                  10250.0                    9000
   2 │     2       225.0        0.0                  10125.0                   11250
   3 │     3       190.0        0.0                  10450.0                   10450
   4 │     4       160.0       20.0                   8320.0                    7280
   5 │     5       225.0       25.0                  10800.0                   10560
   6 │     6       225.0        0.0                  11250.0                   12500

Total cost: $61795.0
Naive production cost: $61040
Cost difference: $155.0
